In [1]:
import numpy as np
np.random.seed(1)

from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Input
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
# define documents
docs = ['king is man',
'a king marrys queen',
'a queen marrys king',
'some unrelated workds'
'queen is woman']
docs

['king is man',
 'a king marrys queen',
 'a queen marrys king',
 'some unrelated workdsqueen is woman']

In [3]:
vocab_size = 50

In [4]:
oh = np.array(one_hot('king queen man woman unrelated', vocab_size)).reshape(-1,1)
oh

array([[15],
       [11],
       [30],
       [ 7],
       [ 7]])

In [5]:
enc = OneHotEncoder()
enc.fit(np.array(range(50)).reshape(-1,1))
oh_enc = enc.transform(oh).toarray()
oh_enc

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
     

In [6]:
encoded_docs = [one_hot(d, vocab_size) for d in docs]
encoded_docs 

[[15, 4, 30], [38, 15, 11, 11], [38, 11, 11, 15], [1, 7, 43, 4, 7]]

In [7]:
max_length = 24
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
padded_docs 

array([[15,  4, 30,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0],
       [38, 15, 11, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0],
       [38, 11, 11, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0],
       [ 1,  7, 43,  4,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0]], dtype=int32)

In [11]:
tuples = np.empty((0, 2))
for padded_doc in padded_docs:
    length = len(padded_doc)
    for i in range(length):
        if padded_doc[i] != 0:
            if i<length-1 & padded_doc[i+1] != 0:
                tuples = np.append(tuples, [[padded_doc[i],padded_doc[i+1]]], axis=0) 
                if i<length-2 & padded_doc[i+2] != 0:
                    tuples = np.append(tuples, [[padded_doc[i],padded_doc[i+2]]], axis=0) 
            if i > 0:
                tuples = np.append(tuples, [[padded_doc[i],padded_doc[i-1]]], axis=0) 
                if i > 1:
                    tuples = np.append(tuples, [[padded_doc[i],padded_doc[i-2]]], axis=0) 

    
print (tuples.shape)
tuples



(40, 2)


array([[ 15.,   4.],
       [ 15.,  30.],
       [  4.,  30.],
       [  4.,  15.],
       [ 30.,   4.],
       [ 30.,  15.],
       [ 38.,  15.],
       [ 38.,  11.],
       [ 15.,  11.],
       [ 15.,  11.],
       [ 15.,  38.],
       [ 11.,  11.],
       [ 11.,  15.],
       [ 11.,  38.],
       [ 11.,  11.],
       [ 11.,  15.],
       [ 38.,  11.],
       [ 38.,  11.],
       [ 11.,  11.],
       [ 11.,  15.],
       [ 11.,  38.],
       [ 11.,  15.],
       [ 11.,  11.],
       [ 11.,  38.],
       [ 15.,  11.],
       [ 15.,  11.],
       [  1.,   7.],
       [  1.,  43.],
       [  7.,  43.],
       [  7.,   4.],
       [  7.,   1.],
       [ 43.,   4.],
       [ 43.,   7.],
       [ 43.,   7.],
       [ 43.,   1.],
       [  4.,   7.],
       [  4.,  43.],
       [  4.,   7.],
       [  7.,   4.],
       [  7.,  43.]])

In [12]:
onehotlabels_x = enc.transform(tuples[:,0].reshape(-1, 1)).toarray()

print  (onehotlabels_x.shape)
print ('---')
print (onehotlabels_x)
print ('---')
print (onehotlabels_x[0])

(40, 50)
---
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
---
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [13]:
onehotlabels_y = enc.transform(tuples[:,1].reshape(-1, 1)).toarray()

print  (onehotlabels_y.shape)
print ('---')
print (onehotlabels_y)
print ('---')
print (onehotlabels_y[0])

(40, 50)
---
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
---
[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [14]:
model = Sequential()

input = Dense(50, input_shape=(50,), activation='relu')
model.add(input)
bottleneck = Dense(2, activation='relu')
model.add(bottleneck)
model.add(Dense(50, activation='softmax'))
# compile the model
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

# fit the model
model.fit(onehotlabels_x, onehotlabels_y, epochs=500, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 102       
_________________________________________________________________
dense_3 (Dense)              (None, 50)                150       
Total params: 2,802
Trainable params: 2,802
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
40/40 [==============================] - 0s 3ms/step - loss: 0.0987 - acc: 0.9800
Epoch 2/500
40/40 [==============================] - 0s 551us/step - loss: 0.0986 - acc: 0.9800
Epoch 3/500
40/40 [==============================] - 0s 109us/step - loss: 0.0984 - acc: 0.9800
Epoch 4/500
40/40 [==============================] - 0s 104us/step - loss: 0.0983 - acc: 0.9800
Epoch 5/500
40/40

In [15]:
model = Sequential()
model.add(input)
model.add(bottleneck)
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['acc'])
test = model.predict(oh_enc)
print (test)


[[  0.          13.04530525]
 [  0.          14.59268951]
 [  0.          10.17055321]
 [  0.           6.23393059]
 [  0.           6.23393154]]


In [16]:
model = Sequential()
model.add(Embedding(1000, 2, input_length=5))
model.compile('rmsprop', 'mse')

input_array = np.random.randint(1000, size=(12, 5)) # 12 documents, 5 words per document
input_array


array([[334, 552, 326, 148, 279],
       [453, 879, 647, 958, 667],
       [957,   8,  28, 339, 566],
       [230, 355, 482,  22, 278],
       [193, 715, 621, 183,  29],
       [647, 521,  78, 207, 585],
       [247, 889, 901, 525, 342],
       [284, 854, 870, 774,  95],
       [ 81, 178, 733, 128,  73],
       [323, 725, 232, 756,  33],
       [825, 751, 193, 208, 126],
       [313, 412, 239, 233,  67]])

In [17]:

output_array = model.predict(input_array)
print (output_array.shape)
print (output_array)

(12, 5, 2)
[[[  4.22293432e-02  -3.46678495e-03]
  [  7.48813152e-04   2.89256610e-02]
  [  1.74377896e-02   1.98974945e-02]
  [  2.56128237e-03   1.63544156e-02]
  [ -3.37575078e-02  -3.63755710e-02]]

 [[  4.17576768e-02  -1.25897042e-02]
  [ -1.34926327e-02  -4.45657484e-02]
  [ -3.22887786e-02  -4.74164635e-03]
  [  2.84376182e-02  -4.81118560e-02]
  [ -7.40225241e-03   1.01623908e-02]]

 [[  3.17674316e-02   2.20933296e-02]
  [  3.72495987e-02  -1.39600635e-02]
  [ -4.59349155e-03   4.25161980e-02]
  [  4.32658084e-02   2.25010253e-02]
  [  4.55187671e-02   1.93335749e-02]]

 [[  1.66166946e-03  -4.26523946e-02]
  [  1.82152651e-02  -4.51023690e-02]
  [  3.78516950e-02   7.98095390e-03]
  [  5.41528314e-03  -2.41282713e-02]
  [  1.37226619e-02   2.54724175e-03]]

 [[ -2.85745747e-02   3.10760848e-02]
  [ -4.69998270e-03  -2.49655247e-02]
  [  3.69708613e-03  -4.27030697e-02]
  [ -4.30753343e-02   9.06718895e-03]
  [ -1.10068209e-02   3.90919708e-02]]

 [[ -3.22887786e-02  -4.74164

In [18]:
# define documents
docs = ['Well done!',
'Good work',
'Great effort',
'nice work',
'Excellent!',
'Really Weak',
'Poor effort!',
'not good',
'poor work',
'Could have done better.']
# define class labels
labels = [1,1,1,1,1,0,0,0,0,0]

In [19]:
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[17, 6], [25, 42], [32, 10], [14, 42], [25], [10, 20], [15, 10], [36, 25], [15, 42], [16, 8, 6, 47]]


In [20]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[17  6  0  0]
 [25 42  0  0]
 [32 10  0  0]
 [14 42  0  0]
 [25  0  0  0]
 [10 20  0  0]
 [15 10  0  0]
 [36 25  0  0]
 [15 42  0  0]
 [16  8  6 47]]


In [21]:
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 89.999998


In [23]:
import numpy as np
np.hstack((model.predict(padded_docs),np.array(labels).reshape(10,1)))

array([[ 0.55897105,  1.        ],
       [ 0.56527364,  1.        ],
       [ 0.54366922,  1.        ],
       [ 0.5653519 ,  1.        ],
       [ 0.55505389,  1.        ],
       [ 0.44329858,  0.        ],
       [ 0.47669604,  0.        ],
       [ 0.47448897,  0.        ],
       [ 0.50290853,  0.        ],
       [ 0.40020174,  0.        ]])